In [67]:
!pip install opendatasets

In [69]:
import opendatasets as od

In [71]:
od.download("https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification")

Skipping, found downloaded files in ".\recyclable-and-household-waste-classification" (use force=True to force download)


In [73]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# Define the path to the main images folder
image_folder = '/content/recyclable-and-household-waste-classification/images/images'

# Traverse the subdirectories and list images
image_paths = []
for root, dirs, files in os.walk(image_folder):
    for file in files:
        # Append full path for each image file
        image_paths.append(os.path.join(root, file))

# Display a few sample images
for i, image_path in enumerate(image_paths[:5]):  # Show the first 5 images
    image = Image.open(image_path)

    # Display the image
    plt.subplot(1, 5, i + 1)
    plt.imshow(image)
    plt.axis('off')
    plt.title(os.path.basename(image_path))

plt.show()

In [75]:
import os
import shutil
import random

# Define paths
dataset_dir = '/content/recyclable-and-household-waste-classification/images/images'
train_dir = '/content/split_dataset/train/images'
val_dir = '/content/split_dataset/val/images'

# Ensure train and val directories exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Process each class in the dataset directory
for class_name in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_path):  # Ensure it’s a directory (class folder)
        # Create class folders in train and val directories
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

        # Collect all image file paths from any subdirectories within this class folder
        images = []
        for root, _, files in os.walk(class_path):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image file extensions
                    images.append(os.path.join(root, file))

        # Shuffle and split images into 80/20 train/val sets
        random.shuffle(images)
        split_idx = int(len(images) * 0.8)
        train_images = images[:split_idx]
        val_images = images[split_idx:]

        # Copy images to train and val directories
        for image_path in train_images:
            dest_path = os.path.join(train_dir, class_name, os.path.basename(image_path))
            shutil.copyfile(image_path, dest_path)

        for image_path in val_images:
            dest_path = os.path.join(val_dir, class_name, os.path.basename(image_path))
            shutil.copyfile(image_path, dest_path)

print("Dataset successfully split into train and val folders!")


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/recyclable-and-household-waste-classification/images/images'

In [11]:
# Load a base configuration for Cascade R-CNN
cfg = Config.fromfile('configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py')

# Set dataset paths
cfg.dataset_type = 'CocoDataset'
cfg.data_root = 'data/recyclable_waste/'

cfg.data.train = dict(
    type='CocoDataset',
    ann_file='data/recyclable_waste/annotations/instances_train.json',
    img_prefix='data/recyclable_waste/images/train/',
    classes=('plastic', 'metal', 'paper', 'glass')  # Define your classes
)

cfg.data.val = dict(
    type='CocoDataset',
    ann_file='data/recyclable_waste/annotations/instances_val.json',
    img_prefix='data/recyclable_waste/images/val/',
    classes=('plastic', 'metal', 'paper', 'glass')
)

cfg.data.test = dict(
    type='CocoDataset',
    ann_file='data/recyclable_waste/annotations/instances_val.json',
    img_prefix='data/recyclable_waste/images/val/',
    classes=('plastic', 'metal', 'paper', 'glass')
)

# Adjust model for the custom dataset (e.g., number of classes)
cfg.model.roi_head.bbox_head[0].num_classes = 4
cfg.model.roi_head.bbox_head[1].num_classes = 4
cfg.model.roi_head.bbox_head[2].num_classes = 4

# Output directory for logs and model checkpoints
cfg.work_dir = './work_dirs/cascade_rcnn_recycling'

# Training settings
cfg.optimizer.lr = 0.02  # Adjust learning rate for small datasets
cfg.lr_config.warmup = None
cfg.log_config.interval = 10  # Log every 10 iterations
cfg.runner.max_epochs = 12  # Number of training epochs
cfg.evaluation.metric = 'bbox'

# Enable GPU training
cfg.gpu_ids = range(1)

# Print updated configuration
print(f'Updated Config:\n{cfg.pretty_text}')

NameError: name 'Config' is not defined

In [ ]:
# Initialize the model with the configuration
model = init_detector(cfg, device='cuda:0')

# Train the model
train_detector(model, cfg)

In [ ]:


# Load the trained model
checkpoint = './work_dirs/cascade_rcnn_recycling/latest.pth'
model = init_detector(cfg, checkpoint, device='cuda:0')

# Run inference on a test image
image_path = 'data/recyclable_waste/images/val/test_image.jpg'
result = inference_detector(model, image_path)

# Visualize results
show_result_pyplot(model, image_path, result, score_thr=0.3)

In [ ]:

from mmdet.datasets import build_dataloader, build_dataset

# Build the test dataset and data loader
test_dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(test_dataset, samples_per_gpu=1, workers_per_gpu=2, dist=False, shuffle=False)

# Run evaluation
results = single_gpu_test(model, data_loader)
print(results)